# Runtime Experiments

These experiments should be run first to ensure the runtimes are measured with ruleset generators running in serial (other notebooks speed up experiments through parallelisation).

In [ ]:
import pandas as pd

from witan_experiments import (is_cached,
                               save_to_cache,
                               load_from_cache,
                               run_experiments)
from witan_experiments.evaluation import (summarise_experiments,
                                          build_metric_df,
                                          display_metric_table)
from witan_experiments.config import prepare_experiment_configs
from witan_experiments.rule_seeders import BlankRS, AccRS
from witan_experiments.rule_generators import (IWSBinaryRG,
                                               WitanRG,
                                               SnubaRG,
                                               SemiSupervisedRG,
                                               ActiveLearningRG,
                                               CbiRG,
                                               HdcRG)
from witan_experiments.labellers import SnorkelLblr
from witan_experiments.models import AnnClf

In [ ]:
ruleset_generators = {
    'Wɪᴛᴀɴ': WitanRG(),
    'Wɪᴛᴀɴ-Core': WitanRG(a=False, o=1),
    'IWS-AS': IWSBinaryRG(acq='AS'),
    'IWS-LSE-AC': IWSBinaryRG(acq='LSE'),
    'Snuba': SnubaRG(),
    'HDC': HdcRG(),
    'CBI': CbiRG(clf=AnnClf()),
    'Semi-supervised': SemiSupervisedRG(),
    'Active learning': ActiveLearningRG(clf=AnnClf(), init_count=0),
}

base_config = dict(
    rngseed=[1],
    labeller=[SnorkelLblr()],
    classifier=[AnnClf()],
)
unseeded_config = {
    **base_config,
    **dict(
        rule_seeder=[BlankRS()],
        ruleset_generator=[
            ruleset_generators['Wɪᴛᴀɴ'],
            ruleset_generators['Wɪᴛᴀɴ-Core'],
            ruleset_generators['IWS-AS'],
            ruleset_generators['IWS-LSE-AC'],
            ruleset_generators['Snuba'],
            ruleset_generators['Semi-supervised'],
            ruleset_generators['Active learning'],
            ruleset_generators['HDC'],
        ],
    ),
}
seeded_config = {
    **base_config,
    **dict(
        rule_seeder=[AccRS()],
        ruleset_generator=[
            ruleset_generators['CBI'],
        ],
    ),
}

full_ic = [25, 100, 200]
min_ic = [25, 100]
dataset_to_ic = {
    'imdb': full_ic,
    'imdb_genre': min_ic,
    'bias_pa': full_ic,
    'bias_pt': min_ic,
    'bias_jp': min_ic,
    'bias_pp': min_ic,
    'amazon': min_ic,
    'yelp': min_ic,
    'plots': min_ic,
    'fakenews': full_ic,
    'binary_dbpedia': min_ic,
    'binary_agnews': full_ic,
    'airline_tweets': min_ic,
    'damage': min_ic,
    'spam': min_ic,
}
dataset_configs = {
    dataset: [
        *prepare_experiment_configs(**unseeded_config, dataset_name=[dataset], interaction_count=ic),
        *prepare_experiment_configs(**seeded_config, dataset_name=[dataset], interaction_count=ic),
    ]
    for dataset, ic in dataset_to_ic.items()
}

In [ ]:
CACHE_KEY = 'runtime-experiments'

if not is_cached(CACHE_KEY):
    dfs = []
    for dataset, configs in dataset_configs.items():
        print(f'\nRunning experiments for: {dataset}')
        dataset_results = run_experiments(
            configs,
            default_workers=2,
            rule_workers=1,
            continue_on_failure=False,
        )
        dfs.append(summarise_experiments(dataset_results, workers=8))
    df = pd.concat(dfs)
    save_to_cache(CACHE_KEY, df)

df = load_from_cache(CACHE_KEY)

In [ ]:
table_df = df[df['interaction_count'].isin([25, 100])].copy()

## Runtime

We compare the runtimes of rule generation methods after 25 and 100 user interactions. We highlight differences in runtime seconds compared to the baseline of `Wɪᴛᴀɴ`.

In [ ]:
runtime_df = build_metric_df(table_df,
                             method='ruleset_generator',
                             metric='rule_gen_wall_secs',
                             labelled_methods=ruleset_generators)
table = display_metric_table(runtime_df, baseline_label='Wɪᴛᴀɴ', small_margin=10, big_margin=60,
                             larger_is_better=False, formatter='{:.1f}')
display(table)
print(table.to_latex(multirow_align='t', convert_css=True))